This should be the final pipeline for the cleaning and preparing the dataset. End result is a data frame, ready to be tossed into a ML model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

Loading the data, and convert all the strings to lower case for uniformity.

In [27]:
df_0 = pd.read_csv("../data/Training_Set_Values.csv")
df_1 = pd.read_csv("../data/Training_Set_labels.csv")
df = df_0.merge(df_1, on='id')
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].map(lambda x: x.lower() if isinstance(x, str) else x)

In [10]:
from sklearn.model_selection import train_test_split

# Replace with your actual target column name
target_col = 'status_group'  # Example: use your actual label column

X = df.drop(columns=[target_col])  # Drop target and ID from features
y = df[target_col]

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)


In [16]:
len(X_test_ids)

14850

In [12]:
X_test_ids = X_test['id'].copy()  # Store before dropping


In [18]:
# Columns to be dropped for the baseline models
columns_to_drop=['id','amount_tsh','num_private','recorded_by','scheme_name',
              'extraction_type_group','extraction_type_class',
              'management','payment_type','quality_group','quantity_group','source','waterpoint_type_group',
              'funder','installer','wpt_name','scheme_management']

X_test = X_test.drop(columns=columns_to_drop)
X_train = X_train.drop(columns=columns_to_drop)

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class GeoContextImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.geo_group_cols = ['subvillage', 'ward', 'lga', 'district_code', 'region_code']
        self.pop_group_cols = ['subvillage', 'ward', 'lga', 'district_code', 'region_code']

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
    
        # Handle subvillage
        if 'subvillage' in X.columns:
            X['subvillage'] = X['subvillage'].replace('', pd.NA)
            X['subvillage'] = X.apply(lambda row: self.fill_subvillage(row, X), axis=1)
    
        # Handle geo columns
        for col in ['latitude', 'longitude']:
            if col in X.columns:
                X[col] = X[col].apply(lambda x: pd.NA if abs(x) < 1e-6 else x)
    
        if 'latitude' in X.columns and 'longitude' in X.columns:
            X = self.fill_missing_geo(X, self.geo_group_cols)
    
        # Handle population
        if 'population' in X.columns:
            X['population'] = X['population'].apply(lambda x: pd.NA if x in [0, 1] else x)
            X = self.fill_missing_population(X, self.pop_group_cols)
    
        # Handle gps_height and construction_year
        if 'gps_height' in X.columns:
            X['gps_height'] = X['gps_height'].apply(lambda x: pd.NA if x == 0 else x)
            X = self.geo_groupwise_fill(X, 'gps_height', self.geo_group_cols)
    
        if 'construction_year' in X.columns:
            X['construction_year'] = X['construction_year'].apply(lambda x: pd.NA if x == 0 else x)
            X = self.geo_groupwise_fill_mode(X, 'construction_year', self.geo_group_cols)
    
        # Simple fill
        if 'wpt_name' in X.columns:
            X['wpt_name'] = X['wpt_name'].fillna('none')
        if 'scheme_management' in X.columns:
            X['scheme_management'] = X['scheme_management'].fillna('other')
    
        # Relabel by coverage (robust)
        for col in ['funder', 'installer', 'scheme_name']:
            if col in X.columns:
                X = self.relabel_by_coverage(X, col, threshold=0.5)
    
        return X.replace({pd.NA: np.nan})

    def fill_subvillage(self, row, df):
        if pd.isna(row['subvillage']):
            ward_mode = df[df['ward'] == row['ward']]['subvillage'].mode()
            if not ward_mode.empty:
                return ward_mode[0]

            lga_mode = df[df['lga'] == row['lga']]['subvillage'].mode()
            if not lga_mode.empty:
                return lga_mode[0]

            district_mode = df[df['district_code'] == row['district_code']]['subvillage'].mode()
            if not district_mode.empty:
                return district_mode[0]

            return 'unknown'
        return row['subvillage']

    def fill_missing_geo(self, df, group_cols):
        for col in group_cols:
            group_means = df.dropna(subset=['latitude', 'longitude']).groupby(col)[['latitude', 'longitude']].mean()

            def fill(row):
                if pd.isna(row['latitude']) or pd.isna(row['longitude']):
                    key = row[col]
                    if key in group_means.index:
                        if pd.isna(row['latitude']):
                            row['latitude'] = group_means.loc[key, 'latitude']
                        if pd.isna(row['longitude']):
                            row['longitude'] = group_means.loc[key, 'longitude']
                return row

            df = df.apply(fill, axis=1)

            if df['latitude'].isna().sum() == 0 and df['longitude'].isna().sum() == 0:
                break

        return df

    def fill_missing_population(self, df, levels):
        for level in levels:
            group_medians = df.dropna(subset=['population']).groupby(level)['population'].median()

            def fill(row):
                if pd.isna(row['population']):
                    key = row[level]
                    if key in group_medians:
                        row['population'] = group_medians.loc[key]
                return row

            df = df.apply(fill, axis=1)
            if df['population'].isna().sum() == 0:
                break

        return df

    def geo_groupwise_fill(self, df, target_col, group_cols):
        for col in group_cols:
            group_medians = df.dropna(subset=[target_col]).groupby(col)[target_col].median()

            def fill(row):
                if pd.isna(row[target_col]):
                    key = row[col]
                    if key in group_medians.index:
                        row[target_col] = group_medians.loc[key]
                return row

            df = df.apply(fill, axis=1)
            if df[target_col].isna().sum() == 0:
                break

        return df

    def geo_groupwise_fill_mode(self, df, target_col, group_cols):
        for col in group_cols:
            group_modes = df.dropna(subset=[target_col]).groupby(col)[target_col].agg(lambda x: x.mode().iloc[0])

            def fill(row):
                if pd.isna(row[target_col]):
                    key = row[col]
                    if key in group_modes.index:
                        row[target_col] = group_modes.loc[key]
                return row

            df = df.apply(fill, axis=1)
            if df[target_col].isna().sum() == 0:
                break

        return df

    def relabel_by_coverage(self, df, column, threshold=0.5, new_label='other'):
        value_counts = df[column].value_counts(normalize=True)
        cumulative = value_counts.cumsum()
        keep_labels = cumulative[cumulative <= threshold].index.tolist()
        df[column] = df[column].apply(lambda x: x if x in keep_labels else new_label)
        return df


In [20]:
# Run the imputer manually to determine available columns
imputer = GeoContextImputer()
X_train_imputed = imputer.fit_transform(X_train.copy())

# Recompute valid column types after imputation
numeric_features = X_train_imputed.select_dtypes(include='number').columns.tolist()
categorical_features = X_train_imputed.select_dtypes(include='object').columns.tolist()

# Now define the transformers as usual
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


In [22]:
from sklearn.ensemble import RandomForestClassifier

full_pipeline = Pipeline([
    ('context_imputer', GeoContextImputer()),  # ← Your custom logic
    ('preprocessor', preprocessor),            # ← ColumnTransformer
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))  # or any model
])


In [23]:
full_pipeline.fit(X_train, y_train)
preds = full_pipeline.predict(X_test)


In [24]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

0.8072727272727273


In [28]:
df_3 = pd.read_csv("../data/Test_Set_Values.csv")

In [29]:
preds_test = full_pipeline.predict(df_3)

In [30]:

# Then build submission
submission = pd.DataFrame({
    "id": df_3["id"].values,
    "status_group": preds_test
})
submission.to_csv("submission.csv", index=False)


Submitting resut on drivendata with testset, public score = 0.76

In [32]:
from sklearn.model_selection import GridSearchCV

params = {
    "classifier__n_estimators": [100, 300],
    "classifier__max_depth": [10, 20, None]
}

grid = GridSearchCV(
    full_pipeline, param_grid=params, cv=3, scoring='accuracy', n_jobs=-1
)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_


In [33]:
print(best_model)

Pipeline(steps=[('context_imputer', GeoContextImputer()),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gps_height', 'longitude',
                                                   'latitude', 'region_code',
                                                   'district_code',
                                                   'population',
                                                   'construction_year']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                          

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

import xgboost as xgb
import lightgbm as lgb

import pandas as pd

# Automatically select column types
numeric_features = X.select_dtypes(include='number').columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

# Preprocessing for numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Full column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb

# Define classifiers
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Extra Trees": ExtraTreesClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "LightGBM": lgb.LGBMClassifier(random_state=42)
}


In [11]:
from sklearn.pipeline import Pipeline

# Build pipelines
pipelines = {
    name: Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ]) for name, model in models.items()
}


In [12]:
from sklearn.metrics import classification_report
import pandas as pd

results = {}

for name, pipeline in pipelines.items():
    print(f"Training {name}...")
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = report["weighted avg"]


Training Decision Tree...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
results_df = pd.DataFrame(results).T
import ace_tools as tools; tools.display_dataframe_to_user(name="Model Evaluation Results", dataframe=results_df)
